## Combine the bankruptcy and sentiment analysis models

In [1]:
# Import dependencies
import tensorflow as tf 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
# Model improvement and Evaluation 
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score

In [3]:
# import bankruptcy and sentiment analysis models
bankruptcy_model = tf.keras.models.load_model('model_checkpoints/model4/ckpt_200')
going_concern_model = tf.saved_model.load('trained_models/going_concern_sentiment_model')

In [ ]:
# preprocess data for sentiment analysis model

In [ ]:
# Load and preprocess the data
X_text = preprocess_text(going_concern_report_text)
X_numeric = preprocess_numeric(numeric_data)
X_numeric = X_numeric.reshape(1, -1)  # Reshape to match expected input shape of bankruptcy model

# Get the output probabilities from the two models
y_bankruptcy = bankruptcy_model.predict(X_numeric)
y_going_concern = going_concern_model.predict(X_text)

# Concatenate the two outputs into a single input vector
X_combined = tf.concat([y_bankruptcy, y_going_concern], axis=1)

# Create a new model to predict the impact of the going-concern report
impact_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the impact model
impact_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Make predictions using the impact model
impact_probability = impact_model.predict(X_combined)

# Print the predicted impact probability
print('Impact probability:', impact_probability[0])